##Import library needed

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

#Create Spark Session

In [3]:
#Create a spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/02/20 00:01:30 WARN Utils: Your hostname, Bowens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.118 instead (on interface en0)
23/02/20 00:01:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 00:01:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/20 00:01:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("fhv_tripdata_2019-01.csv")
print("Here is our inferred schema:")
df.printSchema()

Here is our inferred schema:
root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [4]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00001|2019-01-01 00:30:00|2019-01-01 02:51:55|        null|        null|   null|                B00001|
|              B00001|2019-01-01 00:45:00|2019-01-01 00:54:49|        null|        null|   null|                B00001|
|              B00001|2019-01-01 00:15:00|2019-01-01 00:54:52|        null|        null|   null|                B00001|
|              B00008|2019-01-01 00:19:00|2019-01-01 00:39:00|        null|        null|   null|                B00008|
|              B00008|2019-01-01 00:27:00|2019-01-01 00:37:00|        null|        null|   null|                B00008|
|              B00008|2019-01-01 00:48:0

In [5]:
df.createOrReplaceTempView("tripData")

In [6]:
sqlDF = spark.sql("SELECT DISTINCT dispatching_base_num,  pickup_datetime, dropoff_datetime FROM \
tripData where PULocationID is not null and DOLocationID ='264' and dispatching_base_num= 'B02182' \
GROUP BY dispatching_base_num,  pickup_datetime, dropoff_datetime ORDER BY pickup_datetime ASC")
sqlDF.show()

+--------------------+---------------+----------------+
|dispatching_base_num|pickup_datetime|dropoff_datetime|
+--------------------+---------------+----------------+
+--------------------+---------------+----------------+



In [7]:
df_distinct = spark.sql("SELECT DISTINCT SR_Flag from tripData where PULocationID IS NOT NULL and DOLocationID is NOT NULL ")
df_distinct.show()

+-------+
|SR_Flag|
+-------+
|     26|
|     27|
|     12|
|     22|
|   null|
|      1|
|     13|
|      6|
|     16|
|      3|
|     20|
|      5|
|     19|
|     15|
|      9|
|     17|
|      4|
|      8|
|     23|
|      7|
+-------+
only showing top 20 rows



In [8]:
sqlDF.write.parquet("spark_write_parquet.parquet")

In [22]:
df_parquet = spark.read.parquet("spark_write_parquet.parquet")
df_parquet.show()

+--------------------+-------------------+-------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|
+--------------------+-------------------+-------------------+
|              B02182|2020-04-01 00:15:44|2020-04-01 00:43:22|
|              B02182|2020-04-01 04:22:56|2020-04-01 04:30:10|
|              B02182|2020-04-01 04:44:10|2020-04-01 05:06:28|
|              B02182|2020-04-01 05:04:45|2020-04-01 05:11:29|
|              B02182|2020-04-01 05:16:59|2020-04-01 05:49:06|
|              B02182|2020-04-01 05:23:25|2020-04-01 05:32:03|
|              B02182|2020-04-01 05:39:37|2020-04-01 05:42:09|
|              B02182|2020-04-01 05:46:15|2020-04-01 06:17:27|
|              B02182|2020-04-01 05:57:29|2020-04-01 06:22:49|
|              B02182|2020-04-01 05:59:38|2020-04-01 06:07:01|
|              B02182|2020-04-01 06:14:17|2020-04-01 06:36:56|
|              B02182|2020-04-01 06:33:18|2020-04-01 07:13:58|
|              B02182|2020-04-01 06:41:25|2020-04-01 07

In [27]:
sqlTripData = spark.sql("SELECT dispatching_base_num, PUlocationID, DOlocationID, Affiliated_base_number \
FROM tripData WHERE PULocationID IS NOT NULL and DOLocationID is NOT NULL AND pickup_datetime >= '2020-04-01' \
AND pickup_datetime <='2020-04-03' GROUP BY dispatching_base_num, PUlocationID, DOlocationID, Affiliated_base_number \
ORDER BY dispatching_base_num")
sqlTripData.show()

+--------------------+------------+------------+----------------------+
|dispatching_base_num|PUlocationID|DOlocationID|Affiliated_base_number|
+--------------------+------------+------------+----------------------+
|              B00001|         264|         264|                B00001|
|              B00008|         264|         264|                B00008|
|              B00009|         264|         264|                B00009|
|     B00021         |         260|         226|       B00021         |
|     B00021         |          82|          82|       B00021         |
|     B00021         |          82|         129|       B00021         |
|     B00021         |          70|         129|       B00021         |
|     B00021         |         173|          70|       B00021         |
|     B00021         |         129|         138|       B00021         |
|     B00021         |          56|          82|       B00021         |
|     B00021         |         129|         173|       B00021   